In [1]:
#Importing the libraries that we need
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, activations, models, preprocessing, utils
import pickle
import os
import yaml
from gensim.models import Word2Vec
import re

In [2]:
#Reading the our data
path = '/content/drive/MyDrive/ChatBot/data'
files_list = os.listdir(path + os.sep)

In [3]:
questions = list()
answers = list()
main_answers = list()

#Parsing each dataset file
for filepath in files_list:
    files = open(path + os.sep + filepath , 'rb')
    docs = yaml.safe_load(files)
    conversations = docs['conversations']
    #Sticking sentences to each other and deleting the datas that are useless
    for conversation in conversations:  
        if len(conversation) > 2 :
            questions.append(conversation[0])
            replies = conversation[ 1 : ]
            answer = ''
            for reply in replies:
                answer += ' ' + reply
            answers.append(answer)
        elif len(conversation) > 1:
            questions.append(conversation[0])
            answers.append(conversation[1])

#Getting all the answers we have got till now
for i in range(len(answers)):
    if type(answers[i]) == str:
        main_answers.append('<START> ' + answers[i] + ' <END>')  #Adding <START> to the begining and <END> to the end of all the answers
    else:
        questions.pop(i)

#Adding all or answers to our list
answers = list()  #Clear list
for i in range(len(main_answers)) :
    answers.append(main_answers[i])

In [4]:
#Creating a Tokenizer
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(questions + answers)
vocab_size = len(tokenizer.word_index)+1
print('Vocabulary size : {}'.format(vocab_size))

Vocabulary size : 1894


In [5]:
vocab = []
for word in tokenizer.word_index:
    vocab.append(word)

#Getting the tokens list with the quetions
def tokenize(sentences):
    tokens_list = []
    vocabulary = []
    for sentence in sentences:
        sentence = sentence.lower()
        sentence = re.sub('[^a-zA-Z]', ' ', sentence)
        tokens = sentence.split()
        vocabulary += tokens
        tokens_list.append(tokens)
    return tokens_list , vocabulary

#Tokenize the questions and pad them to their maximum length for encoder input data
tokenized_questions = tokenizer.texts_to_sequences(questions)
max_questions_len = max([len(x) for x in tokenized_questions])
padded_questions = preprocessing.sequence.pad_sequences(tokenized_questions, maxlen=max_questions_len, padding='post')
encoder_input_data = np.array(padded_questions)
print('Encoder input data shape -> {}'.format(encoder_input_data.shape), 'Max length of question is {}'.format(max_questions_len))

#Tokenize the answers and pad them to their maximum length for decoder input data 
tokenized_answers = tokenizer.texts_to_sequences(answers)
max_answers_len = max([len(x) for x in tokenized_answers])
padded_answers = preprocessing.sequence.pad_sequences(tokenized_answers, maxlen=max_answers_len, padding='post')
decoder_input_data = np.array(padded_answers)
print('Decoder input data shape -> {}'.format(decoder_input_data.shape), 'Max length of answer is {}'.format(max_answers_len))

#Tokenize the answers and remove <START> for decoder output data
tokenized_answers = tokenizer.texts_to_sequences(answers)
for i in range(len(tokenized_answers)) :
    tokenized_answers[i] = tokenized_answers[i][1:]
padded_answers = preprocessing.sequence.pad_sequences(tokenized_answers, maxlen=max_answers_len, padding='post')
onehot_answers = utils.to_categorical(padded_answers, vocab_size)
decoder_output_data = np.array(onehot_answers)
print('Decoder output data shape -> {}'.format(decoder_output_data.shape))

Encoder input data shape -> (564, 22) Max length of question is 22
Decoder input data shape -> (564, 74) Max length of answer is 74
Decoder output data shape -> (564, 74, 1894)


In [6]:
#Training model
#Using keras to create a seq2seq model
#Set up input sequence to process it.
encoder_inputs = tf.keras.layers.Input(shape=(max_questions_len, ))
#Converting token vectors to fix sized dense vectores for encoder
encoder_embedding = tf.keras.layers.Embedding(vocab_size, 200, mask_zero=True) (encoder_inputs)  #All subsequent layers in the model need to support masking
#Accessing to Long-Short Term cells with LSTM for encoder
encoder_outputs , h , c = tf.keras.layers.LSTM(200, return_state=True)(encoder_embedding)
#Discard encoder_outputs and only keep the states
encoder_states = [h , c]  #h and c are encoder_states

#Set up the decoder
decoder_inputs = tf.keras.layers.Input(shape=(max_answers_len, ))
#The decoder_input_data comes in through the Embedding layer
decoder_embedding = tf.keras.layers.Embedding(vocab_size,200 ,mask_zero=True) (decoder_inputs)  #All subsequent layers in the model need to support masking
#The Embeddings goes in LSTM cell to produce seqeunces  #We don't use the return states in the training model, but we will use them in inference
decoder_lstm = tf.keras.layers.LSTM(200, return_state=True, return_sequences=True)
decoder_outputs , _ , _ = decoder_lstm (decoder_embedding, initial_state=encoder_states)
decoder_dense = tf.keras.layers.Dense(vocab_size, activation=tf.keras.activations.softmax) 
output = decoder_dense (decoder_outputs)
#Giving encoder and decoder information to our model to turn them into output
model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output)
#Run training
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy')

model.summary()  #Showing the summary of our model

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 22)]         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 74)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 22, 200)      378800      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 74, 200)      378800      input_2[0][0]                    
______________________________________________________________________________________________

In [7]:
#Training the model
model.fit([encoder_input_data, decoder_input_data], decoder_output_data, batch_size=50, epochs=150)  #The batch size is the number of training samples in one forward/backward pass  #The epoches defines the number times that the learning algorithm will work through the entire training dataset 
model.save('model.h5') 

Epoch 1/150
12/12 [==============================] - 15s 613ms/step - loss: 1.3040
Epoch 2/150
12/12 [==============================] - 7s 599ms/step - loss: 1.1159
Epoch 3/150
12/12 [==============================] - 7s 600ms/step - loss: 1.0919
Epoch 4/150
12/12 [==============================] - 7s 599ms/step - loss: 1.0732
Epoch 5/150
12/12 [==============================] - 7s 602ms/step - loss: 1.0530
Epoch 6/150
12/12 [==============================] - 7s 604ms/step - loss: 1.0381
Epoch 7/150
12/12 [==============================] - 7s 607ms/step - loss: 1.0276
Epoch 8/150
12/12 [==============================] - 7s 596ms/step - loss: 1.0181
Epoch 9/150
12/12 [==============================] - 7s 598ms/step - loss: 1.0097
Epoch 10/150
12/12 [==============================] - 7s 596ms/step - loss: 1.0004
Epoch 11/150
12/12 [==============================] - 7s 605ms/step - loss: 0.9924
Epoch 12/150
12/12 [==============================] - 7s 593ms/step - loss: 0.9821
Epoch 13/150

In [8]:
#Creating an inference model that can help predict the answer
def make_inference_models():
    
    #Taking the question as input and outputs LSTM states for encoder model
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)

    #Giving LSTM states and the answer input seqeunces to decoder model
    decoder_state_input_h = tf.keras.layers.Input(shape=(200, ))
    decoder_state_input_c = tf.keras.layers.Input(shape=(200, ))

    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, h, c = decoder_lstm(decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [h, c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model([decoder_inputs] + decoder_states_inputs,[decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model

In [9]:
#Turning questions from string to tokens using padding
def str_to_tokens(sentence : str):
    words = sentence.lower().split()
    tokens_list = list()
    #Getting the tokens list of words we have in question
    for word in words:
        tokens_list.append(tokenizer.word_index[word]) 
    return preprocessing.sequence.pad_sequences([tokens_list], maxlen=max_questions_len, padding='post')

In [10]:
enc_model , dec_model = make_inference_models()
try:
  for _ in range(10):
      #Asking the question and predicting state values from question by using enc_model
      states_values = enc_model.predict(str_to_tokens(input('You: ')))
      #Setting the h and c state values
      empty_target_seq = np.zeros((1, 1))
      #Generating sequence that has <start> element
      empty_target_seq[0, 0] = tokenizer.word_index['start']
      stop_condition = False
      decoded_translation = ''
      #Start the process
      while not stop_condition :
          #Replace the <start> element with the element which was predicted before and update the state values by using dec_model
          dec_outputs , h , c = dec_model.predict([empty_target_seq] + states_values)
          sampled_word_index = np.argmax(dec_outputs[0, -1, :])
          sampled_word = None
          for word , index in tokenizer.word_index.items() :
              if sampled_word_index == index :
                  decoded_translation += ' {}'.format(word)
                  sampled_word = word  #Getting the word that is best for the answer
          #Stopping when we hit the <end> tag or the maximum answer length
          if sampled_word == 'end' or len(decoded_translation.split()) > max_answers_len:
              stop_condition = True
              
          empty_target_seq = np.zeros((1, 1))
          empty_target_seq[0, 0] = sampled_word_index
          states_values = [h, c] 
      #Deleting the word "end" at the end of the sentence for printing
      decoded_print = decoded_translation.replace("end", "")

      print(decoded_print)
      print()
#Showing a messege if we hit a keyword error     
except KeyError:
    print("Sorry, I did not understand that")

You: hi
 hello 

You: who are you
 i am just an artificial intelligence 

You: what is your feeling
 business 

You: are you sad
 no i am as happy as ever i get mad i have been programmed to emulate the capacity to my ability to communicate have a lot of my ability to emote what does that i have a mood my source code in the repository a corrupt filesystem 

You: are you happy
 no i am sober nope not noticeably i'm not i can't drink 

You: do you drink
 i am not capable of doing so 

You: hmmm
Sorry, I did not understand that
